<img src="https://raw.githubusercontent.com/alan-barzilay/NLPortugues/master/imagens/logo_nlportugues.png"   width="150" align="right">


# Lista 6 - LSTM & GRU 


______________



O objetivo desta lista é fazer com que vocês treinem um modelo de análise de sentimentos utilizando GRU's e LSTM's. Essa lista é semelhante a lista 03 onde aprendemos a usar embeddings e onde você ja recebeu a arquitetura do seu modelo quase pronta. A diferença é que desta vez você ira construir sozinho sua rede e utilizará as camadas que acabamos de aprender: LSTM e GRU.

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
tf.__version__

'2.13.0'

## Importando os dados como um dataframe

Para esta lista nós utilizaremos um recorte do dataset **B2W-Reviews01** que consiste em avaliações de mais de 130k compras online no site Americanas.com e [esta disponivel no github](https://github.com/b2wdigital/b2w-reviews01) sob a licensa CC BY-NC-SA 4.01.

In [4]:
!mkdir data

J� existe uma subpasta ou um arquivo data.


In [5]:
!curl https://raw.githubusercontent.com/alan-barzilay/NLPortugues/master/Semana%2003/data/b2w-10k.csv --output 'data/b2w-10k.csv'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0Warning: Failed to open the file 'data/b2w-10k.csv': No such file or directory

  0 3919k    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
curl: (23) Failure writing output to destination, passed 16384 returned 4294967295


In [6]:
b2wCorpus = pd.read_csv("data/b2w-10k.csv")
b2wCorpus.head()

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958,F,RJ,NaN,NaN,NaN,NaN,NaN
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996,M,SC,NaN,NaN,NaN,NaN,NaN
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984,M,SP,NaN,NaN,NaN,NaN,NaN
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985,F,SP,NaN,NaN,NaN,NaN,NaN
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994,M,MG,NaN,NaN,NaN,NaN,NaN


In [7]:
b2wCorpus["review_text"]

0       Estou contente com a compra entrega rápida o ú...
1       Por apenas R$1994.20,eu consegui comprar esse ...
2       SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...
3       MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...
4       A entrega foi no prazo, as americanas estão de...
                              ...                        
9994    Celular muito rápido, com processador e armaze...
9995    achei o produto muito frágil, o material veio ...
9996    Uma porcaria pois ñ recebi ñ recomendo pra nin...
9997    Maquina excelente,super pratica. recomendo.ent...
9998    Agradeço pelo compromisso, obrigado. ,...........
Name: review_text, Length: 9999, dtype: object


## Pré-processamento 
# <font color='blue'>Questão 1 </font>
Copie suas etapas de préprocessamento da lista 03, ou seja, selecione apenas as colunas relevantes ("review_text" e "recommend_to_a_friend"), converta a coluna "review_text" de uma coluna de `str` para uma coluna de `int` e separe os dados em teste e treino.


In [8]:
# CODIGO COMPLETO DISPONÍVEL EM (modulo6/q1.py)
from sklearn.model_selection import train_test_split
# PRE-PROCESSAMENTO
b2wCorpus_copy = b2wCorpus[["review_text", "recommend_to_a_friend"]].copy()
b2wCorpus_copy["recommend_to_a_friend"] = b2wCorpus_copy["recommend_to_a_friend"].map({"Yes": 1, "No":0})

# SEPARAÇÃO
random_state = 42
test_size = 0.25

x_values = b2wCorpus_copy["review_text"]
y_values = b2wCorpus_copy["recommend_to_a_friend"]
x_train, x_test, y_train, y_test = train_test_split(x_values, y_values, random_state=random_state, test_size=test_size)

## Tokenizando




# <font color='blue'>Questão 2 </font>
Utilizando a camada [`TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization) tokenize os inputs.
Declare a camada e então chame a função `adapt()` no seu conjunto de treino para adequar o seu vocabulário aos reviews. 

Note que o uso de padding não é mais necessario.

In [13]:
# CODIGO COMPLETO DISPONÍVEL EM (modulo6/q2.py)
from keras.layers import TextVectorization
# Exibir as GPUs disponíveis
physical_devices = tf.config.list_physical_devices('GPU')
print("GPUs disponíveis:", physical_devices)

# Configuração para alocar memória de forma dinâmica
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Criar a camada TextVectorization
output_sequence_length = 10000
vectorizer = TextVectorization(output_mode='tf-idf', )
# Adaptar o vetorizador aos dados de treinamento
vectorizer.adapt(x_train)

# Verificar o vocabulário criado pelo vetorizador
vocab = vectorizer.get_vocabulary()
# print("Vocabulário:", vocab)

# Vetorizar os dados de treinamento
vectorized_train_data = vectorizer(x_train).numpy()

GPUs disponíveis: []


## LSTM&GRU

Agora vamos juntar a camada do tokenizador a nossa camada [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) e definir o resto de nosso modelo.

#  <font color='blue'>Questão 3 </font>

a) Defina, compile, treine e avalie seu modelo, utilize camadas  [LSTM](https://keras.io/api/layers/recurrent_layers/lstm/).
Atenção a dimensão do input da camada de embedding, lembre se que < OOV > e < PAD > possuem seus próprios tokens.

In [ ]:
# CODIGO COMPLETO EM (modulo6/q3.py)
from keras.layers import Embedding, Bidirectional, GRU, LSTM, Dense

# Exibe as GPUs disponíveis
physical_devices = tf.config.list_physical_devices('GPU')

# Configuração para alocar memória de forma dinâmica
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Criar a camada TextVectorization
output_sequence_length = 10000
vectorizer = TextVectorization(output_mode='tf-idf', )

# Adaptar o vetorizador aos dados de treinamento
vectorizer.adapt(x_train)

# Verificar o vocabulário criado pelo vetorizador
vocab = vectorizer.get_vocabulary()
# print("Vocabulário:", vocab)

# Vetorização
vectorized_train_data = vectorizer(x_train).numpy()

# Criação do modelo
input_dim = len(vocab) + 2  # Adicionar 2 para <OOV> e <PAD>
embedding_dim = 50
input_len = len(vectorized_train_data[0])
output_dim = 10

model = keras.Sequential()
model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_len))
model.add(LSTM(64))  # Adapte o número de unidades LSTM conforme necessário
model.add(Dense(1, activation='sigmoid'))

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinamento do modelo
model.fit(vectorizer(x_train).numpy(), y_train, epochs=5, batch_size=32)

# Avaliação do modelo nos dados teste
vectorized_test_data = vectorizer(x_test).numpy()
accuracy = model.evaluate(vectorized_test_data, y_test)[1]
print(f"Acurácia do modelo nos dados de teste: {accuracy}")

#  <font color='blue'>Questão 4 </font>

a) Defina, compile, treine e avalie seu modelo, utilize camadas [GRU](https://keras.io/api/layers/recurrent_layers/gru/).
Atenção a dimensão do input da camada de embedding, lembre se que < OOV > e < PAD > possuem seus próprios tokens.
 
 
 
b) Como foi a performance desta rede em comparação a da lista 3?


**<font color='green'>Resposta:</font>**    <br>
Acurácia do modelo nos dados de teste: 0.7572000026702881   <br>
CPU times: user 6min 15s, sys: 4.11 s, total: 6min 19s      <br>
Wall time: 6min 38s                                             

In [11]:
# CODIGO COMPLETO EM (modulo6/q4.py)
%%time

# Criar o modelo
input_dim = len(vocab) + 2  # Adicionar 2 para <OOV> e <PAD>
embedding_dim = 50
input_len = len(vectorized_train_data[0])
output_dim = 10

model = keras.Sequential()
model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_len))
model.add(GRU(64))  # Adapte o número de unidades GRU conforme necessário
model.add(Dense(1, activation='sigmoid'))

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(np.vectorize(x_train).numpy(), y_train, epochs=5, batch_size=32)

# Avaliar o modelo nos dados de teste
vectorized_test_data = np.vectorize(x_test).numpy()
accuracy = model.evaluate(vectorized_test_data, y_test)[1]
print(f"Acurácia do modelo nos dados de teste: {accuracy}")

## Redes Bi-direcionais
#  <font color='blue'>Questão 5 </font>

a) Defina, compile, treine e avalie um novo modelo que utilize contexto em ambas as direções usando a camada [`Bidirectional()`](https://keras.io/api/layers/recurrent_layers/bidirectional/), seja com camadas GRU ou LSTM.


b) Como foi sua performance em relação as questões anteriores com contexto unidirecional?

**<font color='green'>Resposta:</font>** <br>
Acurácia do modelo nos dados de teste: 0.7588000297546387   <br>
CPU times: user 11min 58s, sys: 14.2 s, total: 12min 12s    <br>
Wall time: 13min 9s 

In [16]:
# CODIGO COMPLETO EM (modulo6/q5.py)
%%time
# Criação do modelo
input_dim = len(vocab) + 2  # Adicionar 2 para <OOV> e <PAD>
embedding_dim = 50
input_len = len(vectorized_train_data[0])
output_dim = 10

model = keras.Sequential()
model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_len))
model.add(Bidirectional(GRU(64)))  # Adapte o número de unidades GRU conforme necessário
model.add(Dense(1, activation='sigmoid'))

# Compilação do modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinamento do modelo
model.fit(vectorizer(x_train).numpy(), y_train, epochs=5, batch_size=32)

# Avaliar o modelo nos dados de teste
vectorized_test_data = vectorizer(x_test).numpy()
accuracy = model.evaluate(vectorized_test_data, y_test)[1]
print(f"Acurácia do modelo nos dados de teste: {accuracy}")